## Attempt to get Tripifier working

In [1]:
%load_ext autoreload
%autoreload 2

from typing import List, Dict
from google.transit import gtfs_realtime_pb2 as gtfs_rt
from protobuf_to_dict import protobuf_to_dict
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from datetime import datetime
import sys, os

from pandas.io.json import json_normalize
from itertools import groupby

# TODO better way to import module?
sys.path.insert(0, os.path.realpath('/content/jupyter/mta-accessibility/notebooks/routing'))
import GTFS_Utils as gu

# TODO fork tripifier eventually
# Import local version of tripifier
sys.path.insert(0, os.path.abspath('/content/jupytermta-accessibility/gtfs-tripify/'))
import gtfs_tripify as gt
from gtfs_tripify import tripify as tp

- Tripifier takes a long time to create logbooks
- Even logbooks is still kind of messy data

In [3]:
%%time

#'/content/jupyter/20190601/gtfs_ace_20190601_062731.gtfs' contains trip_id 038900_E..S
#file_names[9]

file_names = gu.get_file_names('/content/jupyter/20190601', 'ace', '20190601')
feed = gtfs_rt.FeedMessage()
print(f"Num Files: {len(file_names)}")

feed_dicts = []
for file_path in file_names[:100]:    
    with open(file_path, 'rb') as file:        
        raw_str = file.read()
        feed.ParseFromString(raw_str)
        dt = tp.dictify(feed)
    feed_dicts.append(dt)    
        
log_book, timestamps, errors, _ = gt.logify(feed_dicts)
print(f"Log book size {len(log_book)}")

Num Files: 3301
Log book size 3754
CPU times: user 2min 44s, sys: 1.28 s, total: 2min 45s
Wall time: 2min 45s


In [4]:
%%time
df = pd.DataFrame()
for key, vdf in log_book.items():
    vdf['unique_trip_id'] = key
    df = df.append(vdf)

CPU times: user 3min 46s, sys: 10.1 s, total: 3min 56s
Wall time: 3min 56s


In [5]:
df.head()

,trip_id,route_id,action,minimum_time,maximum_time,stop_id,latest_information_time,unique_trip_id
0,031350_E..S,E,EN_ROUTE_TO,1.559427e+09,NaN,G05S,1559426525,083300_E..S_1559408369
1,031350_E..S,E,EN_ROUTE_TO,1.559427e+09,NaN,G06S,1559426525,083300_E..S_1559408369
2,031350_E..S,E,EN_ROUTE_TO,1.559427e+09,NaN,G07S,1559426525,083300_E..S_1559408369
3,031350_E..S,E,EN_ROUTE_TO,1.559427e+09,NaN,F05S,1559426525,083300_E..S_1559408369
4,031350_E..S,E,EN_ROUTE_TO,1.559427e+09,NaN,F06S,1559426525,083300_E..S_1559408369


In [ ]:
df[df.unique_trip_id == '089200_A..S_1559412004']
#df[df.unique_trip_id == '086900_E..S_1559413045']
#df.groupby('action')['trip_id'].count()

In [53]:
df = pd.read_pickle('tripified.pkl')
stop_df = pd.read_csv('~/jupyter/google_transit/stops.txt')
df = df.merge(stop_df[['stop_id', 'stop_name']], on='stop_id')
#df[df.route_id == 'A'].sort_values('latest_information_time')
df['minimum_time'] = pd.to_datetime(df.minimum_time, unit='s')
df['maximum_time'] = pd.to_datetime(df.maximum_time, unit='s')
df['latest_information_time'] = pd.to_datetime(df.latest_information_time, unit='s')

In [57]:
df[df.route_id == 'E'].sort_values('unique_trip_id')

,trip_id,route_id,action,minimum_time,maximum_time,stop_id,latest_information_time,unique_trip_id,stop_name
10570,020833_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G07N,2019-06-01 08:23:16,020833_E..N_1559377396,Jamaica - Van Wyck
20381,020833_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G05N,2019-06-01 08:23:16,020833_E..N_1559377396,Jamaica Center - Parsons/Archer
10895,020833_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G06N,2019-06-01 08:23:16,020833_E..N_1559377396,Sutphin Blvd - Archer Av - JFK Airport
19628,020833_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,F06N,2019-06-01 08:23:16,020833_E..N_1559377396,Kew Gardens - Union Tpke
8630,021300_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,E01S,2019-06-01 08:23:16,021300_E..S_1559377396,World Trade Center
10463,022801_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G07N,2019-06-01 08:23:16,022801_E..N_1559377396,Jamaica - Van Wyck
19894,022801_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G15N,2019-06-01 08:23:16,022801_E..N_1559377396,65 St
19984,022801_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G12N,2019-06-01 08:23:16,022801_E..N_1559377396,Grand Av - Newtown
20268,022801_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G05N,2019-06-01 08:23:16,022801_E..N_1559377396,Jamaica Center - Parsons/Archer
20030,022801_E..N,E,STOPPED_OR_SKIPPED,2019-06-01 08:23:16,2019-06-01 08:23:16,G11N,2019-06-01 08:23:16,022801_E..N_1559377396,Woodhaven Blvd


array(['031350_E..S', '029400_E..S', '029350_E..S', '050900_E..S',
       '033300_E..S', '036500_E..S', '042500_E..S', '041300_E..S',
       '074900_E..S', '047300_E..S', '038900_E..S', '043700_E..S',
       '040100_E..S', '027350_E..S', '046100_E..S', '035300_E..S',
       '044900_E..S', '060500_E..S', '089300_E..S', '043950_E..S',
       '048500_E..S', '037700_E..S', '102500_E..S', '059366_E..S',
       '054500_E..S', '067700_E..S', '068910_E..S', '052100_E..S',
       '061700_E..S', '049700_E..S', '095300_E..S', '055700_E..S',
       '106138_E..S', '058021_E..S', '072505_E..S', '025450_E..S',
       '076100_E..S', '077311_E..S', '092900_E..S', '064141_E..S',
       '056935_E..S', '121800_E..S', '061730_E..S', '078500_E..S',
       '071300_E..S', '090500_E..S', '070100_E..S', '112100_E..S',
       '066480_E..S', '095345_E..S', '023300_E..S', '066500_E..S',
       '094100_E..S', '044911_E..S', '083300_E..S', '096525_E..S',
       '088100_E..S', '079700_E..S', '073701_E..S', '086900_E.

In [93]:
dfe = df[df.route_id == 'E']
dfe = dfe[(dfe.stop_id.isin(['E01S', 'A34S']))]
dfe['count'] = dfe.groupby('unique_trip_id')['stop_name'].transform('count')
dfe = dfe[dfe['count'] == 2]

#dfe[dfe['count'] == 2].sort_values('unique_trip_id')

dfe.sort_values('unique_trip_id')


,trip_id,route_id,action,minimum_time,maximum_time,stop_id,latest_information_time,unique_trip_id,stop_name,count
7707,023300_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 14:52:15,2019-06-01 14:52:15,A34S,2019-06-01 14:52:15,023300_E..S_1559377396,Canal St,2
8469,023300_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 14:52:15,2019-06-01 14:52:15,E01S,2019-06-01 14:52:15,023300_E..S_1559377396,World Trade Center,2
8022,025450_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 08:56:17,2019-06-01 08:56:17,A34S,2019-06-01 08:56:17,025450_E..S_1559379377,Canal St,2
8557,025450_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 08:56:17,2019-06-01 08:56:17,E01S,2019-06-01 08:56:17,025450_E..S_1559379377,World Trade Center,2
8531,027350_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 09:11:05,2019-06-01 09:11:05,E01S,2019-06-01 09:11:05,027350_E..S_1559380265,World Trade Center,2
7956,027350_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 09:11:05,2019-06-01 09:11:05,A34S,2019-06-01 09:11:05,027350_E..S_1559380265,Canal St,2
8532,027350_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 09:18:09,2019-06-01 09:18:09,E01S,2019-06-01 09:18:09,027350_E..S_1559380689,World Trade Center,2
7957,027350_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 09:18:09,2019-06-01 09:18:09,A34S,2019-06-01 09:18:09,027350_E..S_1559380689,Canal St,2
8390,027350_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 14:14:49,2019-06-01 14:14:49,E01S,2019-06-01 14:14:49,027350_E..S_1559380999,World Trade Center,2
7450,027350_E..S,E,STOPPED_OR_SKIPPED,2019-06-01 11:34:27,2019-06-01 14:14:49,A34S,2019-06-01 14:51:15,027350_E..S_1559380999,Canal St,2


In [106]:
dfe['diff'] = dfe.groupby(['unique_trip_id'])['minimum_time'].transform('diff')
dfe[~dfe['diff'].isnull()]

,trip_id,route_id,action,minimum_time,maximum_time,stop_id,latest_information_time,unique_trip_id,stop_name,count,diff


In [121]:
#dfe.groupby(['unique_trip_id', 'minimum_time']).count()
#dfe[dfe.unique_trip_id == '027350_E..S_1559380999']['minimum_time'].diff()
d = dfe.groupby(['unique_trip_id'])['minimum_time'].diff().reset_index()


## Scratch for Debugging Tripifier

In [195]:
# for d in feed_dicts:
#     for e in d['entity']:
#         if 'trip_update' in e and e['trip_update']['trip']['trip_id'] == '038900_E..S':
#             print(1)

d2 = {'id': '000017E', 'trip_update': {'trip': {'trip_id': '038900_E..S', 'start_date': '20190601', 'route_id': 'E'}, 'stop_time_update': [{'stop_id': 'G05S', 'arrival': 1559384940, 'departure': 1559384940}, {'stop_id': 'G06S', 'arrival': 1559385030, 'departure': 1559385030}, {'stop_id': 'G07S', 'arrival': 1559385180, 'departure': 1559385180}, {'stop_id': 'F05S', 'arrival': 1559385270, 'departure': 1559385270}, {'stop_id': 'F06S', 'arrival': 1559385390, 'departure': 1559385390}, {'stop_id': 'F07S', 'arrival': 1559385480, 'departure': 1559385480}, {'stop_id': 'G08S', 'arrival': 1559385600, 'departure': 1559385600}, {'stop_id': 'G14S', 'arrival': 1559386020, 'departure': 1559386020}, {'stop_id': 'G21S', 'arrival': 1559386380, 'departure': 1559386380}, {'stop_id': 'F09S', 'arrival': 1559386470, 'departure': 1559386470}, {'stop_id': 'F11S', 'arrival': 1559386650, 'departure': 1559386650}, {'stop_id': 'F12S', 'arrival': 1559386740, 'departure': 1559386740}, {'stop_id': 'D14S', 'arrival': 1559386860, 'departure': 1559386860}, {'stop_id': 'A25S', 'arrival': 1559387265, 'departure': 1559387265}, {'stop_id': 'A27S', 'arrival': 1559387700, 'departure': 1559387700}, {'stop_id': 'A28S', 'arrival': 1559387760, 'departure': 1559387760}, {'stop_id': 'A30S', 'arrival': 1559387880, 'departure': 1559387880}, {'stop_id': 'A31S', 'arrival': 1559387970, 'departure': 1559387970}, {'stop_id': 'A32S', 'arrival': 1559388090, 'departure': 1559388090}, {'stop_id': 'A33S', 'arrival': 1559388180, 'departure': 1559388180}, {'stop_id': 'A34S', 'arrival': 1559388270, 'departure': 1559388270}, {'stop_id': 'E01S', 'arrival': 1559388390, 'departure': 1559388390}]}, 'type': 'trip_update'}            
#print(feed_dicts[0])
mock_dict = {'header': {'gtfs_realtime_version': '1.0', 'timestamp': 1559384851}, 'entity': [d2]}
log_book, timestamps, errors, action_logs = gt.logify([mock_dict])

df = pd.DataFrame()
for key, vdf in log_book.items():
    vdf['unique_trip_id'] = key
    df = df.append(vdf)

In [ ]:
log_book, timestamps, errors = gt.logify(feed_dicts)
df = pd.DataFrame()
for key, vdf in log_book.items():
    vdf['unique_trip_id'] = key
    df = df.append(vdf)
    
df[df.trip_id == '038900_E..S'] 

In [196]:
# TODO is there a difference between logifying each file seperately vs load all in memory at once


In [128]:
%%time 
log_book, timestamps, errors = gt.logify(feed_dicts)

action logs [Empty DataFrame
Columns: [trip_id, route_id, information_time, action, stop_id, time_assigned]
Index: []]
CPU times: user 13.5 s, sys: 63.3 ms, total: 13.6 s
Wall time: 13.6 s


In [115]:
df = pd.DataFrame()
for key, vdf in log_book.items():
    vdf['unique_trip_id'] = key
    df = df.append(vdf)
    
#df.latest_information_time = pd.to_datetime(df.latest_information_time, unit='s')
#df.minimum_time = pd.to_datetime(df.minimum_time, unit='s')
#df['minimum_time'] = df['minimum_time'].apply(lambda x: '%.10f' % x)
df.head()

,trip_id,route_id,action,minimum_time,maximum_time,stop_id,latest_information_time,unique_trip_id
0,045900_H..N,H,EN_ROUTE_TO,1559389327.000,nan,H15N,1559389327,045900_H..N_1559388516
1,045900_H..N,H,EN_ROUTE_TO,1559389327.000,nan,H14N,1559389327,045900_H..N_1559388516
2,045900_H..N,H,EN_ROUTE_TO,1559389327.000,nan,H13N,1559389327,045900_H..N_1559388516
3,045900_H..N,H,EN_ROUTE_TO,1559389327.000,nan,H12N,1559389327,045900_H..N_1559388516
4,045900_H..N,H,EN_ROUTE_TO,1559389327.000,nan,H04N,1559389327,045900_H..N_1559388516


In [88]:
rows = []
for d in feed_dicts:
    for entry in d['entity']:
        if 'trip_update' in entry:
            trip_id = entry['trip_update']['trip']['trip_id']
            if trip_id == '038900_E..S':
                rows += (entry['trip_update']['stop_time_update'])
                
for r in rows:
    print(r)

{'stop_id': 'G05S', 'arrival': 1559384940, 'departure': 1559384940}
{'stop_id': 'G06S', 'arrival': 1559385030, 'departure': 1559385030}
{'stop_id': 'G07S', 'arrival': 1559385180, 'departure': 1559385180}
{'stop_id': 'F05S', 'arrival': 1559385270, 'departure': 1559385270}
{'stop_id': 'F06S', 'arrival': 1559385390, 'departure': 1559385390}
{'stop_id': 'F07S', 'arrival': 1559385480, 'departure': 1559385480}
{'stop_id': 'G08S', 'arrival': 1559385600, 'departure': 1559385600}
{'stop_id': 'G14S', 'arrival': 1559386020, 'departure': 1559386020}
{'stop_id': 'G21S', 'arrival': 1559386380, 'departure': 1559386380}
{'stop_id': 'F09S', 'arrival': 1559386470, 'departure': 1559386470}
{'stop_id': 'F11S', 'arrival': 1559386650, 'departure': 1559386650}
{'stop_id': 'F12S', 'arrival': 1559386740, 'departure': 1559386740}
{'stop_id': 'D14S', 'arrival': 1559386860, 'departure': 1559386860}
{'stop_id': 'A25S', 'arrival': 1559387265, 'departure': 1559387265}
{'stop_id': 'A27S', 'arrival': 1559387700, 'depa

In [29]:


path = '201906/20190601/gtfs_ace_20190601_041956.gtfs'
path2 = '20190601/gtfs_ace_20190601_062632.gtfs'

feed = gtfs_rt.FeedMessage()
feed2 = gtfs_rt.FeedMessage()

raw_str = None
raw_str2 = None

log = None
with open(path, 'rb') as file:
    raw_str = file.read()
with open(path2, 'rb') as file:
    raw_str2 = file.read()

feed.ParseFromString(raw_str)
dt = tp.dictify(feed) 

feed.ParseFromString(raw_str2)
dt2 = tp.dictify(feed)
print(dt['header'])
print(dt2['header'])
#log = gt.logify([dt])
    
#dt1 = protobuf_to_dict(feed1)
#dt2 = protobuf_to_dict(feed2)

#gt.logify([dt1, dt2])    

{'gtfs_realtime_version': '1.0', 'timestamp': 1559377196}
{'gtfs_realtime_version': '1.0', 'timestamp': 1559384792}


In [41]:
import gtfs_tripify as gt
from zipfile import ZipFile
import os

# Update this value with the path to the GTFS-RT rollup on your local machine.
DOWNLOAD_URL = '/content/jupyter/201906.zip'

z = ZipFile(DOWNLOAD_URL)
z.extract('20190601.zip')
z.extract('20190612.zip')

messages = []
# filter out non-GTFS files
for filename in sorted(os.listdir('.')):
    if '.py' not in filename and 'gtfs_7_' in filename:
        with open(filename, 'rb') as f:
            messages.append(f.read())

# build the logbooks
first_logbook, first_logbook_timestamps, _ = gt.logify(messages[:len(messages) // 2])
second_logbook, second_logbook_timestamps, _ = gt.logify(messages[len(messages) // 2:])

# merge the logbooks
logbook = gt.ops.merge_logbooks(
    [(first_logbook, first_logbook_timestamps), (second_logbook, second_logbook_timestamps)]    
)

# save to disk
gt.ops.to_csv(logbook, 'logbook.csv')

({}, {})

'20190629'